In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('Supply_chain_db.db')


# Query to print out all table names in the SQLite database
query = "SELECT name FROM sqlite_master WHERE type='table';"
table_names_df = pd.read_sql_query(query, conn)

# Print the first 10 rows of each table
for table_name in table_names_df['name']:
    print(f"First 10 rows of table {table_name}:")
    query = f"SELECT * FROM `{table_name}` LIMIT 5;"
    df = pd.read_sql_query(query, conn)
    print(df)
    print("\n")

# Close the connection
conn.close()

First 10 rows of table projects:
   project_id     name                                description code  \
0         156  Redwing  Blurred Reality of Infinite Possibilities   MJ   

                                       Project Title  
0  Redwing :: Blurred Reality of Infinite Possibi...  


First 10 rows of table wbs:
      id  project_id  level code      description      wbs_string  \
0  10146         156      1   MJ          Redwing              MJ   
1  10147         156      2  RAY        Menomonie          MJ-RAY   
2  10148         156      3  RRH       Forest Run      MJ-RAY-RRH   
3  10149         156      4  FAE          Waywood  MJ-RAY-RRH-FAE   
4  10150         156      4  PCA  Clyde Gallagher  MJ-RAY-RRH-PCA   

  parent_string  parent_id               WBS Title wbs.1 wbs.2 wbs.3 wbs.4  \
0          None        NaN           MJ :: Redwing    MJ  None  None  None   
1            MJ    10146.0        RAY :: Menomonie    MJ   RAY  None  None   
2        MJ-RAY    10147.0   

### Read WBS

In [1]:
wbs_data = [
    "MJ-VQP-UDS-GEBR-JL",
    "MJ-VQP-UDS-LMOL",
    "MJ-VQP-UDS-LMOL-SZJ",
    "MJ-VQP-UDS-LMOL-EE",
    "MJ-VQP-UDS-DRE",
    "MJ-VQP-UMN",
    "MJ-VQP-UMN-ZPH",
    "MJ-VQP-UMN-ZPH-ZV",
    "MJ-CHP",
    "MJ-CHP-KYU",
    "MJ-CHP-KYU-IMN"
]


In [2]:
from collections import defaultdict

def insert_into_tree(tree, path):
    current = tree
    for part in path:
        if part not in current:
            current[part] = {}
        current = current[part]

def build_tree(wbs_data):
    tree = {}
    for item in wbs_data:
        parts = item.split("-")
        insert_into_tree(tree, parts)
    return tree

def tree_to_dict(tree, name):
    # Convert nested dictionary to a format that can be used with visualization tools
    return {
        "name": name,
        "children": [tree_to_dict(subtree, subname) for subname, subtree in tree.items()] if tree else []
    }

# Build the tree from WBS data
tree_structure = build_tree(wbs_data)

# Convert tree to dictionary for visualization
visual_tree_structure = tree_to_dict(tree_structure, "root")
print(visual_tree_structure)


{'name': 'root', 'children': [{'name': 'MJ', 'children': [{'name': 'VQP', 'children': [{'name': 'UDS', 'children': [{'name': 'GEBR', 'children': [{'name': 'JL', 'children': []}]}, {'name': 'LMOL', 'children': [{'name': 'SZJ', 'children': []}, {'name': 'EE', 'children': []}]}, {'name': 'DRE', 'children': []}]}, {'name': 'UMN', 'children': [{'name': 'ZPH', 'children': [{'name': 'ZV', 'children': []}]}]}]}, {'name': 'CHP', 'children': [{'name': 'KYU', 'children': [{'name': 'IMN', 'children': []}]}]}]}]}


In [3]:
wbs_formatted = pd.read_csv('../data/wbs_source_formatted.csv')
display(wbs_formatted.columns)

Index(['L3', 'L3 Desc', 'L4', 'L4 Desc', 'L5', 'Item', 'Description', 'Level',
       'Responsible (From OBS)', 'Commentary', 'Unnamed: 10', 'wbs-code',
       'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14'],
      dtype='object')

In [16]:
# Import necessary libraries
import pandas as pd
import sqlite3

# Function to create the SQLite database table for Groups
def create_groups_table(dbname):
    with sqlite3.connect(dbname) as conn:
        cursor = conn.cursor()
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS Groups (
            group_id TEXT PRIMARY KEY,
            group_description TEXT
        )
        ''')
        print("Table Groups created successfully.")

def insert_group_data(dataframe, dbname):
    try:
        with sqlite3.connect(dbname) as conn:
            cursor = conn.cursor()
            # Use UPSERT to insert or update existing records
            for _, row in dataframe.iterrows():
                cursor.execute('''
                INSERT INTO Groups (group_id, group_description)
                VALUES (?, ?)
                ON CONFLICT(group_id) DO UPDATE SET
                group_description = excluded.group_description;
                ''', (row['Group ID'], row['Group Description']))
            conn.commit()
            print("Data inserted or updated successfully.")
    except sqlite3.Error as e:
        print(f"An error occurred while inserting or updating data: {e}")


# Function to read the Excel file and populate the database
def read_excel_and_populate_db(filepath, dbname):
    xls = pd.ExcelFile(filepath)
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name)
        print(f"Processing sheet: {sheet_name} with columns: {df.columns}")
        insert_group_data(df, dbname)
    print("Database populated successfully from all sheets.")

# Function to display the contents of the Groups database table
def print_table_contents(dbname):
    with sqlite3.connect(dbname) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM Groups")
        rows = cursor.fetchall()
        print("Contents of Groups table:")
        for row in rows:
            print(row)



In [6]:
xls = pd.ExcelFile(excel_path)
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)
    print(f"Processing sheet: {sheet_name} with columns: {df.columns}")
    display(df.head())

Processing sheet: L0 - Program with columns: Index(['Group ID', 'Group Description'], dtype='object')


,Group ID,Group Description
0,QH,QLD Hydro PHES


Processing sheet: L1 - Projects with columns: Index(['Group ID', 'Group Description'], dtype='object')


,Group ID,Group Description
0,BR,Borumba PHES
1,PB,Pioneer-Burdekin PHES
2,QH,Program / Corporate


Processing sheet: L2 - Phases with columns: Index(['Group ID', 'Group Description'], dtype='object')


...

,Group ID,Group Description
0,A1,QH Costs
1,A2,Surface Infrastructure and Enabling
2,A3,Underground Waterways
3,A4,Underground Access & Caverns
4,A5,Powerhouse Mech & Elec


Processing sheet: L3-L4 - Element with columns: Index(['Group ID', 'Group Description'], dtype='object')


,Group ID,Group Description
0,A1-10,QH Delivery (Staff)
1,A1-20,Project Contractors
2,A1-30,Engineering
3,A1-40,Environment and Approvals
4,A1-50,Community Development and Stakeholder


Processing sheet: L3-L5 - Sub Element  with columns: Index(['Group ID', 'Group Description '], dtype='object')


,Group ID,Group Description
0,A1-10-010,Project (Staff)
1,A1-10-020,Commercial (Staff)
2,A1-10-030,Engineering Assurance (Staff)
3,A1-10-040,Health & Safety (Staff)
4,A1-10-050,Project Controls (Staff)


In [20]:
# Specify your database name and Excel file path
database_name = 'wbs.db'
excel_path = '../data/wbs_code_final.xlsx'  # Make sure to provide the correct path

# Create table, populate the database, and print contents
create_table(database_name)
read_excel_and_populate_db(excel_path, database_name)


Table WBS_Elements created successfully.
Processing sheet: L0 - Program with columns: Index(['Group ID', 'Group Description'], dtype='object')
An error occurred while inserting or updating data: no such table: Groups
Processing sheet: L1 - Projects with columns: Index(['Group ID', 'Group Description'], dtype='object')
An error occurred while inserting or updating data: no such table: Groups
Processing sheet: L2 - Phases with columns: Index(['Group ID', 'Group Description'], dtype='object')
An error occurred while inserting or updating data: no such table: Groups
Processing sheet: L3 - Area with columns: Index(['Group ID', 'Group Description'], dtype='object')
An error occurred while inserting or updating data: no such table: Groups
Processing sheet: L3-L4 - Element with columns: Index(['Group ID', 'Group Description'], dtype='object')
An error occurred while inserting or updating data: no such table: Groups
Processing sheet: L3-L5 - Sub Element  with columns: Index(['Group ID', 'Group 

In [21]:
print_table_contents(database_name)

OperationalError: no such table: Groups